In [1]:
import pandas as pd, numpy as np
from datetime import datetime
pd.set_option('max_colwidth',200)
# import yfinance as yf
import tushare as ts
ts.set_token('2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67')
import time, urllib
import glob
import os

In [2]:
def get_data(code,start,end):
    df = ts.pro_bar(ts_code=code, adj='qfq', start_date=start, end_date=end)
    return df

#获取当前交易日最新的股票代码和简称
def get_code():
    codes = pro.stock_basic(list_status='L').ts_code.values
    return codes


def download_tushare_stocks_data(start, end, ch_db_path, ticker_list):
    count = 1
    for ticker in ticker_list:
        if count%200==0:
            print("=======================Sleeping======================")
            time.sleep(60)
        else:
            if not os.path.exists(ch_db_path+ticker+".csv"):
                print("{} is new, start downloading now...".format(ticker))
                try:
                    data = get_data(ticker)
    #                 print(data)
                    data['trade_date'] = data['trade_date'].astype(str)
                    data['trade_date'] = data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                    data.sort_values("trade_date", inplace = True)
                    data.to_csv(ch_db_path+ticker+".csv", index = False)
                    print("{} data file created: {}".format(ticker, end))
                except Exception as e:
                    print(e)

            else:
                print("Already have data csv for {}".format(ticker))
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")   
                hist_data['trade_date'] = hist_data['trade_date'].astype(str)
                hist_data['trade_date'] = hist_data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                hist_data.to_csv(ch_db_path+ticker+".csv", index = False)
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")
                try:
                    hist_data_last_date = hist_data['trade_date'].values[-1]        
                    if today > hist_data_last_date:
                        print("Needs to update, start updating new data for {} now...".format(ticker))
                        update_start = hist_data_last_date
                        update_end = today
                        with eventlet.Timeout(60,False):
                            try:
                                new_data = get_data(ticker, update_start, update_end)
                                new_data['trade_date'] = new_data['trade_date'].astype(str)
                                new_data['trade_date'] = new_data['trade_date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:])
                                new_data.to_csv(ch_db_path+ticker+".csv", mode='a', header=False, index = False)
                                updated_duplicated_df = pd.read_csv(ch_db_path+ticker+".csv")
                                updated_df = updated_duplicated_df.drop_duplicates("trade_date")
                                updated_df.sort_values("trade_date", inplace = True)
                                updated_df.to_csv(ch_db_path+ticker+".csv", index = False)
                                print("New data updated till today for {}!".format(ticker))
                            except Exception as e:
                                print(e)
            #             print("Timed Out: Update Failed!")
                    else:
                        print("There's no new data to update for {}.".format(ticker))
                except Exception as e:
                    print(e)

        #     print("Data Download/Update for {} is Finished.".format(ticker))
            print("=======================Executed: {}=======================".format(count))
        count+=1
    print("【Updated Finished for today!】")


#设置token
token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
pro = ts.pro_api(token)
codes = get_code()
print("Public Company Numbers by today: ", len(codes))

today = str(datetime.now().date())
start = '20100101'
end = today
ch_db_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
ticker_list = codes

Public Company Numbers by today:  3753


In [ ]:
import eventlet
eventlet.monkey_patch()

download_tushare_stocks_data(start, end, ch_db_path, ticker_list)

Already have data csv for 000001.SZ
Needs to update, start updating new data for 000001.SZ now...


/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '2.6.2' or newer of 'numexpr' (version '2.6.1' currently installed).
  warnings.warn(msg, UserWarning)
/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


New data updated till today for 000001.SZ!
=======================Executed: 1=======================
Already have data csv for 000002.SZ
Needs to update, start updating new data for 000002.SZ now...
New data updated till today for 000002.SZ!
=======================Executed: 2=======================
Already have data csv for 000004.SZ
Needs to update, start updating new data for 000004.SZ now...
New data updated till today for 000004.SZ!
=======================Executed: 3=======================
Already have data csv for 000005.SZ
Needs to update, start updating new data for 000005.SZ now...
New data updated till today for 000005.SZ!
=======================Executed: 4=======================
Already have data csv for 000006.SZ
Needs to update, start updating new data for 000006.SZ now...
New data updated till today for 000006.SZ!
=======================Executed: 5=======================
Already have data csv for 000007.SZ
Needs to update, start updating new data for 000007.SZ now...
New d

New data updated till today for 000059.SZ!
=======================Executed: 43=======================
Already have data csv for 000060.SZ
Needs to update, start updating new data for 000060.SZ now...
New data updated till today for 000060.SZ!
=======================Executed: 44=======================
Already have data csv for 000061.SZ
Needs to update, start updating new data for 000061.SZ now...
New data updated till today for 000061.SZ!
=======================Executed: 45=======================
Already have data csv for 000062.SZ
Needs to update, start updating new data for 000062.SZ now...
New data updated till today for 000062.SZ!
=======================Executed: 46=======================
Already have data csv for 000063.SZ
Needs to update, start updating new data for 000063.SZ now...
New data updated till today for 000063.SZ!
=======================Executed: 47=======================
Already have data csv for 000065.SZ
Needs to update, start updating new data for 000065.SZ now...


New data updated till today for 000416.SZ!
=======================Executed: 84=======================
Already have data csv for 000417.SZ
Needs to update, start updating new data for 000417.SZ now...
New data updated till today for 000417.SZ!
=======================Executed: 85=======================
Already have data csv for 000419.SZ
Needs to update, start updating new data for 000419.SZ now...
New data updated till today for 000419.SZ!
=======================Executed: 86=======================
Already have data csv for 000420.SZ
Needs to update, start updating new data for 000420.SZ now...
New data updated till today for 000420.SZ!
=======================Executed: 87=======================
Already have data csv for 000421.SZ
Needs to update, start updating new data for 000421.SZ now...
New data updated till today for 000421.SZ!
=======================Executed: 88=======================
Already have data csv for 000422.SZ
Needs to update, start updating new data for 000422.SZ now...


...

New data updated till today for 002165.SZ!
=======================Executed: 620=======================
Already have data csv for 002166.SZ
Needs to update, start updating new data for 002166.SZ now...
New data updated till today for 002166.SZ!
=======================Executed: 621=======================
Already have data csv for 002167.SZ
Needs to update, start updating new data for 002167.SZ now...
New data updated till today for 002167.SZ!
=======================Executed: 622=======================
Already have data csv for 002168.SZ
Needs to update, start updating new data for 002168.SZ now...
New data updated till today for 002168.SZ!
=======================Executed: 623=======================
Already have data csv for 002169.SZ
Needs to update, start updating new data for 002169.SZ now...
New data updated till today for 002169.SZ!
=======================Executed: 624=======================
Already have data csv for 002170.SZ
Needs to update, start updating new data for 002170.SZ no

New data updated till today for 002206.SZ!
=======================Executed: 661=======================
Already have data csv for 002207.SZ
Needs to update, start updating new data for 002207.SZ now...
New data updated till today for 002207.SZ!
=======================Executed: 662=======================
Already have data csv for 002208.SZ
Needs to update, start updating new data for 002208.SZ now...
New data updated till today for 002208.SZ!
=======================Executed: 663=======================
Already have data csv for 002209.SZ
Needs to update, start updating new data for 002209.SZ now...
New data updated till today for 002209.SZ!
=======================Executed: 664=======================
Already have data csv for 002210.SZ
Needs to update, start updating new data for 002210.SZ now...
New data updated till today for 002210.SZ!
=======================Executed: 665=======================
Already have data csv for 002211.SZ
Needs to update, start updating new data for 002211.SZ no

New data updated till today for 002247.SZ!
=======================Executed: 702=======================
Already have data csv for 002248.SZ
Needs to update, start updating new data for 002248.SZ now...
New data updated till today for 002248.SZ!
=======================Executed: 703=======================
Already have data csv for 002249.SZ
Needs to update, start updating new data for 002249.SZ now...
New data updated till today for 002249.SZ!
=======================Executed: 704=======================
Already have data csv for 002250.SZ
Needs to update, start updating new data for 002250.SZ now...
New data updated till today for 002250.SZ!
=======================Executed: 705=======================
Already have data csv for 002251.SZ
Needs to update, start updating new data for 002251.SZ now...
New data updated till today for 002251.SZ!
=======================Executed: 706=======================
Already have data csv for 002252.SZ
Needs to update, start updating new data for 002252.SZ no

New data updated till today for 002290.SZ!
=======================Executed: 743=======================
Already have data csv for 002291.SZ
Needs to update, start updating new data for 002291.SZ now...
New data updated till today for 002291.SZ!
=======================Executed: 744=======================
Already have data csv for 002292.SZ
Needs to update, start updating new data for 002292.SZ now...
New data updated till today for 002292.SZ!
=======================Executed: 745=======================
Already have data csv for 002293.SZ
Needs to update, start updating new data for 002293.SZ now...
New data updated till today for 002293.SZ!
=======================Executed: 746=======================
Already have data csv for 002294.SZ
Needs to update, start updating new data for 002294.SZ now...
New data updated till today for 002294.SZ!
=======================Executed: 747=======================
Already have data csv for 002295.SZ
Needs to update, start updating new data for 002295.SZ no

New data updated till today for 002331.SZ!
=======================Executed: 784=======================
Already have data csv for 002332.SZ
Needs to update, start updating new data for 002332.SZ now...
New data updated till today for 002332.SZ!
=======================Executed: 785=======================
Already have data csv for 002333.SZ
Needs to update, start updating new data for 002333.SZ now...
New data updated till today for 002333.SZ!
=======================Executed: 786=======================
Already have data csv for 002334.SZ
Needs to update, start updating new data for 002334.SZ now...
New data updated till today for 002334.SZ!
=======================Executed: 787=======================
Already have data csv for 002335.SZ
Needs to update, start updating new data for 002335.SZ now...
New data updated till today for 002335.SZ!
=======================Executed: 788=======================
Already have data csv for 002336.SZ
Needs to update, start updating new data for 002336.SZ no